<a href="https://colab.research.google.com/github/CarolBw/CB-AI-English-teacher/blob/main/CB_AI_English_teacher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto MVP CB-AI English teacher.
Trabalho de conclusão da Sprint Machine Learning & Analytics.
Pós graduação em Ciencia de Dados, PUC-RJ.



# Visão Geral

Este notebook contém o código para o desenvolvimento de um algoritmo professor de inglês utilizando técnicas de Processamento de Linguagem Natural (PNL) e Aprendizado de Máquina. O objetivo é criar um sistema capaz de entender consultas em inglês e fornecer respostas relevantes e informativas para auxiliar no aprendizado da língua inglesa.



# Preparação do Ambiente:





#### Instalando as dependencias >>

In [1]:
# Instalando as dependencias
!pip install tensorflow transformers nltk pandas scikit-learn matplotlib seaborn wordcloud
!pip install pyspellchecker
!pip install textblob
!pip install transformers pandas
!pip install transformers torch
!pip install transformers sklearn
!pip install accelerate -U
!pip install transformers[torch] -U


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


#### Importando as bibliotecas >>

In [2]:
# Bibliotecas para manipulação e análise de dados
import pandas as pd  # Para manipulação de dados em formato de DataFrame
import numpy as np   # Para operações numéricas eficientes em arrays

# Bibliotecas para plotagem de gráficos
import matplotlib.pyplot as plt  # Para plotagem de gráficos
import seaborn as sns            # Para visualização estatística de dados

# Bibliotecas para processamento de linguagem natural
import nltk                      # Para processamento de linguagem natural
from nltk.corpus import stopwords  # Lista de palavras comuns que serão ignoradas em análises de texto
from nltk.tokenize import word_tokenize, sent_tokenize  # Para tokenização de palavras e sentenças
from nltk.stem import WordNetLemmatizer  # Para lematização de palavras

# Ferramentas adicionais de NLP e tokenização
from transformers import BertTokenizer, TFBertForQuestionAnswering  # Para uso de modelos BERT em NLP

# Bibliotecas para aprendizado de máquina e processamento de modelos
import tensorflow as tf             # Para construção de modelos de aprendizado profundo
from tensorflow import keras       # Para construção de modelos de deep learning
from sklearn.model_selection import train_test_split  # Para divisão de dados em treinamento e teste
from sklearn.feature_extraction.text import TfidfVectorizer  # Para vetorização de texto em TF-IDF

# Biblioteca para manipulação de texto e limpeza
import re                         # Para manipulação de expressões regulares
from collections import Counter  # Para contagem de elementos em uma lista
from wordcloud import WordCloud  # Para criação de nuvens de palavras

# Bibliotecas para comunicação com a web e parsing de HTML
import requests                   # Para fazer requisições HTTP
from bs4 import BeautifulSoup    # Para fazer parsing de HTML
from urllib.parse import urlparse  # Para parsing de URLs

# Biblioteca para interação com banco de dados
import sqlite3                   # Para interação com bancos de dados SQLite

# Biblioteca para interação com o Google Colab
from google.colab import files, drive  # Para interação com o Google Colab

# Biblioteca para correção ortográfica
from spellchecker import SpellChecker  # Para correção ortográfica

# Downloads e preparações necessárias para o NLTK
nltk.download('punkt')      # Downloads adicionais para o NLTK
nltk.download('stopwords')  # Downloads adicionais para o NLTK
nltk.download('wordnet')    # Downloads adicionais para o NLTK

# Bibliotecas para pré-processamento de texto para modelos de deep learning
from keras.preprocessing.text import Tokenizer        # Para tokenização de texto
from tensorflow.keras.preprocessing.sequence import pad_sequences


from textblob import TextBlob  # Para processamento de texto e análise de sentimentos

from sklearn.feature_extraction.text import CountVectorizer  # Para vetorização de texto em contagens de palavras
from sklearn.cluster import KMeans  # Para clustering de dados textuais
from sklearn.metrics import silhouette_score  # Para avaliação de clusters
from sklearn.cluster import DBSCAN, AgglomerativeClustering  # Para clustering hierárquico e DBSCAN
from sklearn.decomposition import PCA           # Para redução de dimensionalidade com PCA
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score  # Para avaliação de clusters

from transformers import Trainer, TrainingArguments
import os
import accelerate

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Coleta e carregamento dos dados

  O conjunto de dados consiste em conteúdo textual de Wikibooks em inglês e português, organizado em capítulos e disponibilizado em formato de texto simples e HTML. Recriei o Dataset selecionando apenas estas duas linguagens a partir do conjunto original que continha o conteúdo em diversas outras linguagens.
  
Conjunto original em:
https://www.kaggle.com/datasets/dhruvildave/wikibooks-dataset/data

In [9]:
# Montar o Google Drive
drive.mount('/content/drive')

# Caminho para o arquivo no Google Drive
caminho_arquivo = '/content/drive/MyDrive/Data-Science_Analytics/Projetos/CB-AI English Teacher/wikibooks_pt_en_version.sqlite'

# Conectar ao banco de dados
conexao = sqlite3.connect(caminho_arquivo)

# Cursor para executar comandos SQL
cursor = conexao.cursor()

# Consulta para listar todas as tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tabelas = cursor.fetchall()

# Exibir o nome de todas as tabelas encontradas
print("Tabelas encontradas no banco de dados:")
for tabela in tabelas:
    print(tabela[0])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Tabelas encontradas no banco de dados:
pt
en


# Análise inicial dos dados

#### Exibição de uma prévia do conteúdo dos Datasets >>

In [ ]:
# Nossas tabelas relevantes
tabela_1= 'pt'
tabela_2= 'en'

# Carregar os dados em Dataframe para manipulaçao
query_1 = f"SELECT * FROM {tabela_1};"
df1 = pd.read_sql_query(query_1, conexao)

query_2 = f"SELECT * FROM {tabela_2};"
df2 = pd.read_sql_query(query_2, conexao)

# Fechar a conexão com o banco de dados original
conexao.close()

# Exibir as primeiras linhas para melhor compreesão do conteúdo
print("\nPrimeiras linhas do DataFrame 'pt':")
print(df1.head(20))
print("\nPrimeiras linhas do DataFrame 'en':")
print(df2.head(20))


Exception ignored in: <function _xla_gc_callback at 0x7eadcc40b880>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Colunas do DataFrame 'pt' e 'en'

title: Título do capítulo ou seção.
url: URL do conteúdo.
abstract: Resumo ou introdução do conteúdo.
body_text: Texto principal do conteúdo.
body_html: HTML do conteúdo principal.

Análise:
title: Útil para identificar o assunto de cada entrada, o que pode ser importante para classificar ou organizar os dados de treinamento por tema ou dificuldade.

url: Embora possa ser útil para rastrear a origem do conteúdo ou verificar a integridade dos dados, provavelmente não será necessária para o treinamento do modelo de linguagem.

abstract: Provavelmente útil, pois oferece uma breve descrição ou introdução ao tópico tratado, o que pode ser uma boa fonte de dados concisos para ensinar definições ou conceitos chave.

body_text: Essencial, pois contém a maior parte do conteúdo educativo. Este texto é vital para o treinamento, pois fornece o contexto completo e o desenvolvimento dos temas abordados.

body_html: Menos útil para o treinamento do modelo diretamente devido às tags HTML, mas pode ser importante para extrair texto formatado ou elementos específicos (como tabelas, listas ou seções destacadas), dependendo de como você deseja processar ou apresentar o material educativo.

#### Analisando a estrutura basica dos dados >>

In [ ]:
# Função para analisar a estrutura basica dos dados

def analise_inicial_datasets(df_pt, df_en):
    # Verificando linhas e colunas
    print("Linhas e colunas da tabela 'pt':", df_pt.shape)
    print("Linhas e colunas da tabela 'en':", df_en.shape)

    # Verificando o comprimento médio dos textos
    len_pt = df_pt['body_text'].apply(len).mean()
    len_en = df_en['body_text'].apply(len).mean()
    print("Comprimento médio dos textos em português:", len_pt)
    print("Comprimento médio dos textos em inglês:", len_en)

    # Analisando a diversidade de temas
    un_topics_pt = df_pt['title'].nunique()
    un_topics_en = df_en['title'].nunique()
    print("Número total de tópicos únicos em português:", un_topics_pt)
    print("Número total de tópicos únicos em inglês:", un_topics_en)

    # Recursos educacionais em inglês
    educational_resources_en = df_en[df_en['title'].str.contains('lesson', case=False)]
    print("Número de recursos educacionais em inglês:", len(educational_resources_en))

    # Diversidade e origem dos dados
    unique_urls_pt = df_pt['url'].nunique()
    unique_urls_en = df_en['url'].nunique()
    print("Número de fontes únicas em português:", unique_urls_pt)
    print("Número de fontes únicas em inglês:", unique_urls_en)



In [ ]:
# Aplicando analise inicial nos Datasets

analise_inicial_datasets(df1, df2)

# Processamento inicial básico

In [ ]:
# Deletar colunas irrelevantes
columns_to_remove = ['body_html', 'url']

df1.drop(columns=columns_to_remove, inplace=True)
df2.drop(columns=columns_to_remove, inplace=True)

# Salvar os dataframes
df1.to_csv('cleaned_df1.csv', index=False)
df2.to_csv('cleaned_df2.csv', index=False)
print('Os arquivos.csv, foram salvos para os conjuntos de dados df1 e df2')



In [ ]:
# Verificar duplicatas e remover se necessário
def check_and_remove_duplicates(df, df_name):
    initial_count = len(df)
    df = df.drop_duplicates()
    if len(df) < initial_count:
        print(f"Duplicatas removidas em {df_name}: {initial_count - len(df)} linhas.")
    else:
        print(f"Nenhuma duplicata encontrada em {df_name}.")
    return df

df1 = check_and_remove_duplicates(df1, "df1")
df2 = check_and_remove_duplicates(df2, "df2")

# Função para limpar e formatar strings
def clean_text(text):
    if pd.isna(text):
        return text
    cleaned_text = ' '.join(text.strip().split())
    return cleaned_text

# Aplicar limpeza em colunas de texto
text_columns = ['body_text', 'abstract', 'title']
for column in text_columns:
    df1[column] = df1[column].apply(clean_text)
    df2[column] = df2[column].apply(clean_text)
    print(f"Coluna '{column}' de df1 e df2 limpa e formatada.")

# Verificar e tratar valores nulos
def check_and_handle_nulls(df, df_name):
    for column in df.columns:
        null_count = df[column].isnull().sum()
        if null_count > 0:
            print(f"Valores nulos encontrados na coluna {column} do {df_name}: {null_count}")
            df[column].fillna('Valor Padrão', inplace=True)
            print(f"Valores nulos na coluna {column} do {df_name} tratados.")
        else:
          print('Não foram encontrados valores nulos')

check_and_handle_nulls(df1, "df1")
check_and_handle_nulls(df2, "df2")

# Salvar os DataFrames limpos
df1.to_csv('cleaned_df1.csv', index=False)
df2.to_csv('cleaned_df2.csv', index=False)
print("DataFrames df1 e df2 foram limpos e salvos com sucesso.")


In [ ]:
# Verificar as primeiras linhas novamente para conferir o processamento inicial
print(df1.head(10))
print(df2.head(10))

# Visualizando os dados:

#### Criando graficos para visualizar a distribuição das principais categorias encontradas >>

In [ ]:
# Função para extrair a categoria principal e subcategoria de cada título
def extract_category(title):
    parts = title.split(':')
    if len(parts) > 1:
        category_parts = parts[1].split('/')
        main_category = category_parts[0].strip()
        sub_category = category_parts[1].strip() if len(category_parts) > 1 else None
        return main_category, sub_category
    else:
        return 'Outros', None

# Aplica a função aos DataFrames para extrair as categorias principais e subcategorias
df1[['main_category', 'sub_category']] = df1['title'].apply(lambda x: pd.Series(extract_category(x)))
df2[['main_category', 'sub_category']] = df2['title'].apply(lambda x: pd.Series(extract_category(x)))

# Cria gráficos de barras para as categorias principais e subcategorias em português e inglês
plt.figure(figsize=(10, 6))
sns.countplot(y='main_category', data=df1, order=df1['main_category'].value_counts().index[:30], palette='viridis')
plt.title('Top 30 Categorias Principais em Português')
plt.xlabel('Frequência')
plt.ylabel('Categorias Principais')
plt.show()

plt.figure(figsize=(10, 6))
sns.countplot(y='main_category', data=df2, order=df2['main_category'].value_counts().index[:30], palette='viridis')
plt.title('Top 30 Categorias Principais em Inglês')
plt.xlabel('Frequência')
plt.ylabel('Categorias Principais')
plt.show()

plt.figure(figsize=(10, 6))
sns.countplot(y='sub_category', data=df1.dropna(), order=df1['sub_category'].value_counts().index[:30], palette='viridis')
plt.title('Top 30 Subcategorias em Português')
plt.xlabel('Frequência')
plt.ylabel('Subcategorias')
plt.show()

plt.figure(figsize=(10, 6))
sns.countplot(y='sub_category', data=df2.dropna(), order=df2['sub_category'].value_counts().index[:30], palette='viridis')
plt.title('Top 30 Subcategorias em Inglês')
plt.xlabel('Frequência')
plt.ylabel('Subcategorias')
plt.show()


#### Visualizar comprimento dos textos >>

Aplicação da transformação logarítmica nos comprimentos dos textos para lidar melhor com a dispersao dos dados,
visto que os textos possuem uma grande diversidade de tamanhos.

In [ ]:
def log_transform(x):
    # Adiciona 1 antes de aplicar o log para evitar o logaritmo de zero.
    return np.log(x + 1)

# Aplica a transformação logarítmica nos comprimentos dos textos para os 2 Datasets
log_lengths_pt = df1['body_text'].apply(len).apply(log_transform)
log_lengths_en = df2['body_text'].apply(len).apply(log_transform)

# Configura o estilo dos gráficos
sns.set(style="whitegrid")

# Histograma para a distribuição do comprimento dos textos em português2
plt.figure(figsize=(7, 3))
sns.histplot(log_lengths_pt, bins=30, color="blue", kde=True)
plt.title('Log Distribuição do Comprimento dos Textos em Português')
plt.xlabel('Log Comprimento dos Textos')
plt.ylabel('Frequência')
plt.show()

print()

# Histograma para a distribuição do comprimento dos textos em inglês.
plt.figure(figsize=(7, 3))
sns.histplot(log_lengths_en, bins=30, color="green", kde=True)
plt.title('Log Distribuição do Comprimento dos Textos em Inglês')
plt.xlabel('Log Comprimento dos Textos')
plt.ylabel('Frequência')
plt.show()



Análise dos histogramas de comprimento dos textos:

Após a transformação logarítmica, os histogramas mostram que os dois conjuntos de dados seguem uma distribuição log-normal, mostrando que textos mais curtos são comuns, com uma cauda longa indicando textos significativamente mais longos. A presença destes textos longos pode indicar materiais mais detalhados ou avançados.

Processamento e transformações:

Segmentar textos mais longos em unidades menores mais gerenciáveis. Utilizar técnicas como truncamento, padding ou divisão de textos longos em várias partes para que o comprimento dos textos de entrada seja consistente.


# Processamento dos Dados:




#### Pré processamento avançado de texto >>

In [ ]:
# Processa o texto para tokenização e limpeza básica.

def preprocess_text(text):
    # Separa pontuações dos tokens sem remover, pois são importantes gramaticalmente.
    text = re.sub(r'([,.!?])', r' \1 ', text)
    # Substitui múltiplos espaços por um.
    text = re.sub(r'\s{2,}', ' ', text)
    tokens = word_tokenize(text)
    processed_text = ' '.join(tokens)
    return processed_text

# Segmenta o texto em sentenças.
def segment_text(text):
    return sent_tokenize(text)

# Codifica textos em sequências numéricas e aplica padding.
def encode_and_pad(texts, max_length=100, vocab_size=30000, oov_token="<OOV>"):
    tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
    return padded_sequences, tokenizer

# Aplica pré-processamento nas colunas especificadas.
def apply_preprocessing(df, text_columns):
    for column in text_columns:
        df[f'cleaned_{column}'] = df[column].apply(preprocess_text)
        df[f'segmented_{column}'] = df[f'cleaned_{column}'].apply(segment_text)
    return df

# Processa dois DataFrames para aplicar pré-processamento de texto.
def process_dataframes(df1, df2, text_column):
    df1 = apply_preprocessing(df1, [text_column])
    df2 = apply_preprocessing(df2, [text_column])
    return df1, df2


text_column = 'body_text'
df1, df2 = process_dataframes(df1, df2, text_column)

print("Processed DataFrame 1:", df1)
print("Processed DataFrame 2:", df2)



In [ ]:
# Adicionar uma coluna 'language' para cada DataFrame antes da concatenação
df1['language'] = 'Portuguese'
df2['language'] = 'English'

# Concatenando os dados
df_combined = pd.concat([df1, df2], ignore_index=True)

# Verificar as primeiras linhas para confirmar a concatenação e a coluna de idioma
print(df_combined.head())


#### Salvando o conjunto de dados totalmente processado >>

In [ ]:
# Salvar df1 e df2 em arquivos CSV
df_combined.to_csv('df_processado_combinado.csv', index=False)
print(' O conjunto de dados combinado, foi salvo no diretório atual')

# Caminho para salvar os arquivos CSV no Google Drive
path_df_combined = '/content/drive/MyDrive/Data-Science_Analytics/Projetos/CB-AI English Teacher/df1_processado_combinado.csv'

df_combined.to_csv(path_df_combined, index=False)
print(f'O conjunto de dados combinado foi salvo em: {path_df_combined}')



# Definição e Treinamento do Modelo:




A escolha GPT e os modelos Transformers, foram devido a questão de desempenho superior em processamento de linguagem natural, capacidade de aprendizado contextual, transferência de aprendizado, e disposição de modelos pré treinados.



#### Divisão do conjunto de dados entre treino e teste >>



In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Carregar os dados processados anteriormente
df = pd.read_csv('/content/drive/MyDrive/Data-Science_Analytics/Projetos/CB-AI English Teacher/df1_processado_combinado.csv')

# Verificar conteúdo
print(df.head(10))

# Dividir os dados em conjuntos de treino e teste, com 20% dos dados para teste e semente de reprodutibilidade
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)



                                               title  \
0   Wikilivros: Livro de receitas/Massa para empadas   
1           Wikilivros: Escotismo no Brasil/O que é?   
2              Wikilivros: Guia do Rugby/Leis/O Maul   
3  Wikilivros: Matemática elementar/Geometria pla...   
4            Wikilivros: Português para estrangeiros   
5              Wikilivros: Estruturas metálicas/Capa   
6  Wikilivros: Mecânica dos fluidos/Exercícios re...   
7  Wikilivros: WikiRPG/Lista de fobias e seu uso ...   
8  Wikilivros: ITIL V3/Estratégia de serviço/Conc...   
9                        Wikilivros: Pintura/Cultura   

                                            abstract  \
0               Existem várias maneiras de preparar:   
1                             * O que é? Imagem:25%.   
2  O Maul acontece quando três jogadores, sendo u...   
3                                    == Paralelas ==   
4                        Português para estrangeiros   
5                               Estruturas Metá

#### Definição do modelo e ajuste dos dados >>

Bibliotecas utilizadas:

* transformers: Biblioteca de modelos de deep learning baseados em arquiteturas de Transformer. Utilizada para processamento de linguagem natural, como tradução automática, geração de texto e análise de sentimentos. Transformers suporta TensorFlow e PyTorch.
* torch: É a biblioteca principal do PyTorch, usada para todas as operações com tensores e deep learning.
* DataLoader, TensorDataset: Ferramentas do PyTorch para manipulação de dados em lotes durante o treinamento.
* pad_sequence: Função do PyTorch para padronizar o comprimento dos tensores, essencial para o processamento em lotes.
* GPT2Tokenizer, GPT2Model: Componentes da biblioteca transformers que fornecem o modelo GPT-2 pré-treinado e seu tokenizador.
* pandas: Biblioteca para manipulação de dados, utilizada para operar com DataFrames.



In [4]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import pandas as pd
from torch.utils.data import Dataset, DataLoader

# Carregar o tokenizador e o modelo GPT-2 com uma 'language modeling head'
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Verificar e definir um token de padding se não estiver presente
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Ajustar o modelo para o novo tokenizer

# Classe personalizada para o Dataset
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        if not isinstance(text, str):
            text = ""  # Assegurar que o texto é uma string
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        labels = encoding['input_ids'].roll(-1, dims=1)
        labels[:, -1] = -100  # Usar um índice de ignorar na última posição
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': labels.squeeze(0)
        }

# Definindo a coluna de texto dos Dataframes
train_texts = train_df['cleaned_body_text'].tolist()
test_texts = test_df['cleaned_body_text'].tolist()

# Criar instâncias do Dataset personalizado
train_dataset = TextDataset(train_texts, tokenizer)
test_dataset = TextDataset(test_texts, tokenizer)

# Criar DataLoader para gerenciar os lotes
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


#### Treinamento do modelo utilizando os dados preparados >>

In [ ]:
from transformers import Trainer, TrainingArguments

# Definição dos argumentos de treinamento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=500,
    load_best_model_at_end=True
)

# Configuração do Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Treinar o modelo
trainer.train()

# Avaliar o modelo
results = trainer.evaluate()
print("Avaliação do modelo:", results)



# Avaliação e ajuste do Modelo:



# Geração de respostas em linguagem natural


# Interação com o Usuário:





Implementação de uma interface de usuário para interação com o algoritmo professor de inglês.



Processamento de consultas do usuário e geração de respostas.

#### Classificar niveis de dificuldade >>

In [ ]:
def classificar_dificuldade(df, coluna_texto='body_text', n_clusters=3):
    """
    Classifica os textos do DataFrame em níveis de dificuldade usando o algoritmo KMeans.

    Parâmetros:
        df (pandas.DataFrame): DataFrame contendo os dados.
        coluna_texto (str): Nome da coluna no DataFrame que contém os textos a serem classificados.
        n_clusters (int): Número de níveis de dificuldade (clusters) desejados.

    Retorna:
        pandas.DataFrame: DataFrame com uma nova coluna 'nivel_dificuldade', indicando o
                          nível de dificuldade de cada texto, com base na clusterização.
    """
    # Configuração do TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

    # Transformação dos textos em uma matriz TF-IDF
    tfidf_matrix = tfidf_vectorizer.fit_transform(df[coluna_texto])

    # Configuração e aplicação do KMeans para classificar os textos em níveis de dificuldade
    kmeans = KMeans(n_clusters=n_clusters, random_state=1)
    df['nivel_dificuldade'] = kmeans.fit_predict(tfidf_matrix)

    # Impressão da distribuição dos níveis de dificuldade
    print(f"Distribuição dos níveis de dificuldade dos textos:\n{df['nivel_dificuldade'].value_counts()}")

    return df



Usando métricas para avaliação do desempenho do modelo


In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2Model
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, classification_report

# Carregar o modelo e o tokenizador do GPT-2
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2Model.from_pretrained(model_name)

# Função para codificar os textos em formato compatível com o modelo
def encode_texts(tokenizer, texts):
    return [tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=512) for text in texts]

# Carregar dados de um arquivo CSV
df = pd.read_csv('path_to_your_data.csv')

# Dividir os dados em conjuntos de treino e teste
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Preparar os dados de texto e seus respectivos rótulos
train_texts = train_df['text_column'].tolist()  # Substitua 'text_column' pelo nome real da coluna de texto
train_labels = train_df['label_column'].tolist()  # Substitua 'label_column' pelo nome real da coluna de rótulo
test_texts = test_df['text_column'].tolist()
test_labels = test_df['label_column'].tolist()

# Codificar os textos
train_encoded = encode_texts(tokenizer, train_texts)
test_encoded = encode_texts(tokenizer, test_texts)

# Configuração da validação cruzada para avaliação robusta
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, train_encoded, train_labels, cv=kf, scoring='accuracy')

# Imprimir a acurácia média obtida pela validação cruzada
print("Acurácia média com validação cruzada:", scores.mean())

# Avaliar o modelo no conjunto de teste
# Assumindo que você tem uma função `predict` para fazer previsões com seu modelo
test_predictions = model.predict(test_encoded)  # Ajuste essa linha conforme necessário

# Calcular e imprimir métricas de desempenho
accuracy = accuracy_score(test_labels, test_predictions)
report = classification_report(test_labels, test_predictions)

print("Acurácia no conjunto de teste:", accuracy)
print("Relatório de classificação no conjunto de teste:\n", report)

# Análise de erros (opcional)
errors = test_labels != test_predictions
error_df = test_df[errors]
print("Exemplos de erros de classificação:", error_df.head())



Análise dos resultados

# Testes e Validação:



Testes do sistema em diferentes cenários e condições.



Validação da eficácia e precisão das respostas geradas.

# Integração e Implantação:



Integração do algoritmo professor de inglês em uma aplicação ou plataforma de ensino de inglês.



Implantação do sistema para uso em produção.